In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import pydicom as pyd
import SimpleITK as sk
import math
import numpy as np
from numpy import savetxt
import torch
import torchio as tio
import itk
import os

In [2]:
class DICOM_reader:
    def __init__(self):
        self.file_path = ""
        self.CT_images = []
    
    def set_path(self, path):
        self.file_path = Path(path)
    
    def extract_images(self):
        file_ids = sk.ImageSeriesReader.GetGDCMSeriesIDs(str(self.file_path))
        file_names = sk.ImageSeriesReader.GetGDCMSeriesFileNames(str(self.file_path), file_ids[0])
        series_reader = sk.ImageSeriesReader()
        series_reader.SetFileNames(file_names)
        image_data = series_reader.Execute()
        self.CT_images = sk.GetArrayFromImage(image_data)
                
    def visualize(self, images, if_all, idx=None):
        if if_all:
            idx = list(range(len(images)))
        num1 = math.ceil(math.sqrt(len(idx)))
        num2 = math.floor(math.sqrt(len(idx)))
        num1 = num1+1 if num1*num2<len(idx) else num1
        fig, axis = plt.subplots(num2, num1, figsize=(20,20))
        slice_counter = 0
        if len(idx)==1:
            axis.imshow(images[idx[slice_counter]], cmap="gray")
        else:
            if num2>1:
                for i in range(num2):
                    for j in range(num1):
                        if slice_counter<len(idx):
                            axis[i][j].imshow(images[idx[slice_counter]], cmap="gray")
                            slice_counter+=1
                        else:
                            break
            else:
                for i in range(num1):
                    axis[i].imshow(images[idx[slice_counter]], cmap="gray")
                    slice_counter+=1
                                
    def get_images(self):
        # self.CT_images = [np.expand_dims(np.array(image.astype("int32")),axis=0) for image in self.CT_images]
        return self.CT_images

In [49]:
file_path = "/Users/wangyangwu/Documents/Maastro/NeuralNets/sample/P0439C0006I1473766/RTDOSE/1.3.6.1.4.1.32722.228.6.334797647287977966709364212462384620209/"
file_ids = sk.ImageSeriesReader.GetGDCMSeriesIDs(str(file_path))
file_names = sk.ImageSeriesReader.GetGDCMSeriesFileNames(str(file_path), file_ids[0])
series_reader = sk.ImageSeriesReader()
series_reader.SetFileNames(file_names)
image_data = series_reader.Execute()
CT_images = sk.GetArrayFromImage(image_data)

In [87]:
image_data.GetMetaData('3004|000E')

RuntimeError: Exception thrown in SimpleITK Image_GetMetaData: /tmp/SimpleITK-build/ITK/Modules/Core/Common/src/itkMetaDataDictionary.cxx:77:
ITK ERROR: Key '3004|000E' does not exist 

In [53]:
CT_images.shape*image_data.GetNumberOfComponentsPerPixel()

(1, 99, 100, 191)

In [24]:
import torch.nn as nn

In [2]:
file_reader = sk.ImageFileReader()
file_reader.SetFileName("/Users/wangyangwu/Documents/Maastro/NeuralNets/sample/P0439C0006I1473766/RTDOSE/1.3.6.1.4.1.32722.228.6.334797647287977966709364212462384620209/1.3.6.1.4.1.32722.228.6.36725526625681761397802081573539292769.dcm")
dose = file_reader.Execute()

In [34]:
d = torch.Tensor(np.stack([sk.GetArrayFromImage(dose).astype("float32")]))

In [87]:
np.max(sk.GetArrayFromImage(dose).astype("float32"))

65535.0

In [77]:
softmaxer = nn.Softmax(dim=0)

In [78]:
e = softmaxer(d)

In [79]:
e.sum()

tensor(1890900.)

In [99]:
r = torch.randn(1,1,3,4,5)

In [100]:
r

tensor([[[[[ 0.9084, -2.1716, -1.5608,  0.9693,  1.0599],
           [-1.3696,  0.8443,  0.5409, -1.7633, -0.6439],
           [ 0.4953,  1.7789, -0.3565, -0.0397, -0.4737],
           [ 1.3160,  0.3967,  1.7767, -0.6830,  1.9536]],

          [[ 0.6027, -0.5257,  0.0117, -1.2943,  0.4264],
           [ 1.2755, -1.1833,  1.2318, -0.3948,  0.0425],
           [-0.2996, -1.2784, -3.0218, -0.2793, -1.0589],
           [-0.4263,  0.9547, -0.4759, -1.3520,  0.1217]],

          [[ 1.5289,  1.6560,  0.2515,  1.3280,  1.7406],
           [-0.6894,  0.3340,  0.3991,  0.8495, -0.4814],
           [-0.7442,  0.0183,  1.7785,  0.0787,  1.0072],
           [-1.7541, -1.1762,  0.4201, -1.4321,  0.0500]]]]])

In [101]:
rr = (r-r.min())/(r.max()-r.min())

In [102]:
rr

tensor([[[[[0.7899, 0.1709, 0.2936, 0.8022, 0.8204],
           [0.3321, 0.7770, 0.7161, 0.2529, 0.4779],
           [0.7069, 0.9649, 0.5357, 0.5994, 0.5121],
           [0.8718, 0.6871, 0.9644, 0.4701, 1.0000]],

          [[0.7285, 0.5017, 0.6097, 0.3472, 0.6931],
           [0.8637, 0.3695, 0.8549, 0.5280, 0.6159],
           [0.5471, 0.3504, 0.0000, 0.5512, 0.3945],
           [0.5217, 0.7992, 0.5117, 0.3356, 0.6318]],

          [[0.9146, 0.9402, 0.6579, 0.8743, 0.9572],
           [0.4688, 0.6745, 0.6876, 0.7781, 0.5106],
           [0.4578, 0.6110, 0.9648, 0.6232, 0.8098],
           [0.2548, 0.3709, 0.6918, 0.3195, 0.6174]]]]])

In [97]:
rr.sum()

tensor(48.4965)

In [82]:
e = softmaxer(r)

In [83]:
e

tensor([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]])